# Prediccion de Default en Prestamos


Para este proyecto utilizaremos un sample de los datos de Lending Club. La idea es predecir si cierto usuario cometera Default basado en informacion que la plataforma recolecta. Esto nos ayudara a mejorar la metodologia/pipeline de prestamo.


# Descripcion



Contiene los prestamos de esta plataforma:

    periodo 2007-2017Q3.
    887mil observaciones, sample de 100mil
    150 variables
    Target: loan status



# Objetivo

Realizar un ETL y un EDA

## ETL

0. Limpia los datos de tal manera que al final del ETL queden en formato `tidy`.
1. Asegurate de cargar y leer los datos
2. Crea una tabla donde se guarde el nombre de la columna y el tipo de dato: (`column_name`,   `type`).
3. Asegurate de pensar cual es el tipo de dato correcto. Porque elejiste strig/object o float o int?. No hay respuestas incorrectas como tal, pero tienes que justificar tu decision.
4. Maneja missings o nans de la manera adecuada. Justifica cada decision







## EDA

0. Preparar lo datos para un pipeline de datos
1. Quitar columnas inservibles 
2. Imputar valores
3. Mantener replicabildiad y reproducibilidad

**No olvides anotar tus justificaciones en celdas para recordar cuando te toque explicarlo.** Puedes agregar el numero de celdas que necesites para poner tu explicacion y el codigo, solo manten la estructura.

# ETL

In [1]:
import pandas as pd
import numpy as np
import re


Vas a obtener 2 errores, solucionalo con los visto en clase.  
Tip: Se arreglan con argumentos adicionales de la funcion `read_csv`  
Documentacion: https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html 

In [2]:
loans = pd.read_csv('https://github.com/sonder-art/fdd_prim_2023/blob/main/codigo/pandas/LoansData_sample.csv.gz?raw=true', sep=',', compression='gzip', encoding='latin-1')

print(loans)

       Unnamed: 0        id  member_id  loan_amnt  funded_amnt  \
0               0  38098114        NaN    15000.0      15000.0   
1               1  36805548        NaN    10400.0      10400.0   
2               2  37842129        NaN    21425.0      21425.0   
3               3  37612354        NaN    12800.0      12800.0   
4               4  37662224        NaN     7650.0       7650.0   
...           ...       ...        ...        ...          ...   
99995       99995  22454240        NaN     8400.0       8400.0   
99996       99996  11396920        NaN    10000.0      10000.0   
99997       99997   8556176        NaN    30000.0      30000.0   
99998       99998  24023408        NaN     8475.0       8475.0   
99999       99999  24023398        NaN    25000.0      25000.0   

       funded_amnt_inv        term  int_rate  installment grade  ...  \
0              15000.0   60 months     12.39       336.64     C  ...   
1              10400.0   36 months      6.99       321.08     A

/var/folders/81/3z5tmwbd7sq0s2qmqph5c0c80000gn/T/ipykernel_24567/1129007760.py:1: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  loans = pd.read_csv('https://github.com/sonder-art/fdd_prim_2023/blob/main/codigo/pandas/LoansData_sample.csv.gz?raw=true', sep=',', compression='gzip', encoding='latin-1')


In [3]:
#Limpiando los datos
#Eliminando columnas inecesarias 
loans.drop("Unnamed: 0", axis=1, inplace=True) #Al descomprimir los datos se crea una columna inecesaria
loans.drop("member_id", axis=1, inplace=True)#Es redundante tener la columna member_id ya que es lo mismo que la columna id



In [4]:
#Capitalizando datos
#Capitaliando valores de columnas de tipo texto
valores_cap = ["emp_title", "home_ownership", "verification_status", "purpose", "title"] #Lista de columnas que se van a convertir a string
loans[valores_cap] = loans[valores_cap].apply(lambda x: x.capitalize() if isinstance(x, str) else x)
loans[valores_cap]

,emp_title,home_ownership,verification_status,purpose,title
0,MANAGEMENT,RENT,Source Verified,debt_consolidation,Debt consolidation
1,Truck Driver Delivery Personel,MORTGAGE,Not Verified,credit_card,Credit card refinancing
2,Programming Analysis Supervisor,RENT,Source Verified,credit_card,Credit card refinancing
3,Senior Sales Professional,MORTGAGE,Verified,car,Car financing
4,Technical Specialist,RENT,Source Verified,debt_consolidation,Debt consolidation
...,...,...,...,...,...
99995,Software Engineer,MORTGAGE,Source Verified,debt_consolidation,Debt consolidation
99996,Assistant Professor of English,RENT,Not Verified,debt_consolidation,Debt consolidation
99997,Facilities Tech 3,RENT,Verified,debt_consolidation,Debt consolidation
99998,Records Examiner Analyst,RENT,Verified,debt_consolidation,Debt consolidation


In [5]:
'''para el caso de la columna emp_lenght voy a sustuir el nombre por emp_lenght (years) para tener unicamente valores numericos en la columna 
En el caso de los empleados que tengan menos de un año no se tomaran en cuenta por lo que sus valores seran 0
En el caso de empleados que tienen mas de 10 años la antiguedad se le tomara igual a los que solo tienen 10 años '''
#Cambiando nombre de columna
loans.rename(columns={'emp_length': 'emp_lenght(years)'}, inplace=True)



#Eliminando years de las columnas usando una regular expresion
loans['emp_lenght(years)'] = loans['emp_lenght(years)'].str.replace(r'\s*year(?:s)?$', '', regex=True)

#Intercambiando valores 
loans['emp_lenght(years)'].replace({"< 1": "0", "10+":"10" }, inplace=True)

#Pasando el valor a double
loans['emp_lenght(years)'] = pd.to_numeric(loans['emp_lenght(years)'], errors= 'coerce')




In [11]:
'''En este bloque de codigo volveremos a modificar el nombre de algunas columnas 
ademas de su contenido. Lo que buscamos es que en este dataframe todo este en la misma
unidad de tiempo, en este caso años'''

loans.rename(columns={'term': 'term(years)'}, inplace=True)

#Usaremos una regex para eliminar months
#loans['term(years)'] = loans['term(years)'].str.replace(r'\bmonths\b', '', regex=True)


#Pasando los datos a double 
loans['term(years)'] = pd.to_numeric(loans['term(years)'], errors= 'coerce')

#conviertiendo los meses a años 
#loans['term(years)'] = loans['term(years)']/ 12



float64


In [18]:
'''En este bloque haremos lo mismo algo similar pero para los zipcode '''

#Quitando xx 
#loans['zip_code'] = loans['zip_code'].str.replace(r'xx$', '', regex=True) 

#Pasando los datos a numericos 
loans['zip_code'] = pd.to_numeric(loans['zip_code'], errors= 'coerce')



int64


In [38]:
'''Para facilitar la lectura de datos hay columnas completamente vacias y otras parcialmente vacias
Las que esten completamente vacias se eliminaran para facilitar la lectura '''

#resultado = loans.loc[:, 'columna2':'columna4']


# Función para verificar si al menos un elemento en cada columna es distinto de NaN
def verificar_nans(df, columnas):
    return df[columnas].notnull().any()

columnas_a_verificar = ["annual_inc_joint", "dti_joint",  "verification_status_joint"]
resultados = verificar_nans(loans, columnas_a_verificar)
#print(resultados)

c_f_2 =loans.loc[:,"open_acc_6m": "all_util"]
res= c_f_2.columns.tolist()
res1= verificar_nans(loans, res)
#print(res1) Eliminar 

c_f_3= loans.loc[:,"inq_fi": "inq_last_12m"]
res2= c_f_3.columns.tolist()
res11= verificar_nans(loans, res2)
#print(res11) Eliminar

c_f_4= loans.loc[:,"revol_bal_joint": "sec_app_mths_since_last_major_derog"]
res3= c_f_4.columns.tolist()
res22= verificar_nans(loans, res3)
#print(res22) Eliminar 

c_f_5= loans.loc[:,"hardship_type": "hardship_last_payment_amount"]
res4= c_f_5.columns.tolist()
res33= verificar_nans(loans, res4)
print(res33)




revol_bal_joint                        False
sec_app_fico_range_low                 False
sec_app_fico_range_high                False
sec_app_earliest_cr_line               False
sec_app_inq_last_6mths                 False
sec_app_mort_acc                       False
sec_app_open_acc                       False
sec_app_revol_util                     False
sec_app_open_act_il                    False
sec_app_num_rev_accts                  False
sec_app_chargeoff_within_12_mths       False
sec_app_collections_12_mths_ex_med     False
sec_app_mths_since_last_major_derog    False
dtype: bool


## Tabla (column_name, type)

Revisa el metodo pd.DataFrame.dtypes. https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dtypes.html 

In [ ]:
column_types = loans["term"].dtype
print(column_types)

## Cargar descripcion de columnas

La siguiente tabla tiene una descripcion del significado de cada columna

In [ ]:


datos_dict = pd.read_excel(
    'https://resources.lendingclub.com/LCDataDictionary.xlsx')
datos_dict.columns = ['feature', 'description']


In [ ]:
datos_dict

### Pickle

Crea codigo para **guardar** y **cargar** el DataFrame de `datos_dict` creada en las celdas anteriores en formato **pickle**

In [ ]:
# COdigo guardar

In [ ]:
# Codigo para cargar

## Tipos de Datos

Realiza las transformaciones o casteos (casting) que creas necesarios a tus datos de tal manera que el typo de dato sea adecuado. Al terminar recrea la tabla `column_types` con los nuevos tipos.

No olvides anotar tus justificaciones para recordar cuando te toque explicarlo.

In [ ]:
# Manejos de tipos 1
# Tu codigo aqui

In [ ]:
# Manejos de tipos 2
# Tu codigo aqui


In [ ]:
# column_types =


## Manejo de NaNs o missings

Maneja los datos de tipos missing. Elije una estrategia adecuada dependiendo del tipo de dato que le asignaste a la columna.


Crea codigo para **guardar** y **cargar** un archivo JSON en el que se guarde la `estrategia` y `valor` que utilizaste para **imputar**. Por ejemplo: Si hay una columna que se llama `columna 3` y utilizaste la estrategia de imputacion de media, y existe otra llamada `columna 4` y  elegiste la palabra 'missing' el JSON debera contener:  
  
 `{'columna 3':{'estrategia':'mean', 'valor':3.4}, 'columna 4':{'estrategia':'identificador', 'valor':'missing'}}`  

 De tal manera que para cada columna que tenga un metodo de imputacion apunte a otro diccionario donde el **key** `estrategia` describa de manera sencilla el metodo, y el **key** `valor` el valor usado. En general:   
 `{'nombre de la columna':{'estrategia':'descripcion de estrategia', 'valor':'valor utilizado'}}`. 
 

De utilizar mas de un metodo puedes anidarlos en una lista  
  `[{...},{...}]`.  

Incluso si la columna utilizada no sufrio imputacion, es necesario que la agregues al JSON.

La idea es que cualquier otra persona pueda cargar el el archivo JSON con tu funcion, entender que hiciste y replicarlo facilmente. No existe solo una respuesta correcta, pero tendras que justificar y explicar tus deciciones.

### Imputacion

In [ ]:
# Tu codigo aqui

In [ ]:
# TUcodigo aqui

### Codigo para salvar y cargar JSONs